In [ ]:
# Print issues List created above
# Get json file
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
from IPython.display import display, Math, HTML, Markdown
import datetime  
import pytz
from dateutil.relativedelta import relativedelta
import textwrap
import json

display(HTML("<style>.container { width:310mm !important; }</style>"))
with open('ReposIssuesList.json') as f:
    data = f.read()
reposIssuesList = json.loads(data)
with open('MilestonesList.json') as f:
    data = f.read()
milestonesList = json.loads(data)
#
# To Create WebPdf use:  
#     jupyter nbconvert .\GitHubReportMd.ipynb --to webpdf --no-input --output-dir='../docs' --output MyGitHubReport
#

In [ ]:
OPEN_ISSUES_TEMPLATE = """
\hspace {10mm} \\normalsize {\\textbf{TITLE - #NBR}}  \\Huge {\\phantom{l}} \\\[3pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\ \\\ 
\hspace {10mm} \\bbox[5pt]  {\small {TASKS }} \\\[5pt] 
\hspace {10mm} \small { LABELS } \\\ 
"""
ISSUE_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[1pt] 
"""
CLOSED_ISSUE_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE - Closed on CDATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[1pt] 
\hspace {10mm} \small { LABELS } \\\[1pt] 
"""
LABELS_TEMPLATE = """
\\begin{array}{l l} 
\\textit{Comments:} &  COMM 
\\end{array}  
"""
STATE_TEMPLATE = """
\\begin{array}{l r} 
 \\small { \\color{red} { \\textsf{Open:}}} & OPEN \\\ 
 \\small { \\color{green} {\\textsf{Closed:}}} & CLOSED \\\ 
\\end{array}  
"""
TASKS_TEMPLATE = """
\\begin{array}{l l} 
\\textit{Tasks:} & \\hspace {10mm} TASKS 
\\end{array} 
"""
# print('\nIssues Closed Last Week')
closedTitle = """<div style=\"page-break-after: always;\"></div>
<hr style="height:5px;border-width:0;color:green;background-color:green">
$ \hspace {20mm} \\normalsize \\color{green} 
{\\textbf{Summary of Issues Closed During Last 7 Days (Including Merged Branches)}} \\\[1pt]  $ 
<hr style="height:5px;border-width:0;color:green;background-color:green">
"""

# print('\nIssues Opened Last Week')
openedTitle = """<div style=\"page-break-after: always;\"></div>
<hr style="height:5px;border-width:0;color:red;background-color:red">
$ \hspace {50mm} \\normalsize \\color{red} 
{\\textbf{Summary of Issues Opened During Last 7 Days}} \\\[1pt]  $ 
<hr style="height:5px;border-width:0;color:red;background-color:red">
"""
reportTitle = \
"""\hspace {40mm} \Large {\\textbf{TRUCCR Milestones/Issues Report}} \\\[1pt]
\hspace {80mm} \\small {\\textsf{Date: TDATE  }   }   \\\[20pt] """


In [ ]:
issuesList = []
for repo in reposIssuesList:
    for issue in repo['issues']:
        issueDict = {}
        issueDict['repo_name'] = repo['repo_name']
        issueDict['issue_title'] = issue['title']
        issueDict['issue_number'] = issue['number']
        issueDict['date_opened'] = issue['created_at']
        issueDict['date_closed'] = issue['closed_at']
        issueDict['state'] = issue['state']
        issueDict['body'] = issue['body']
        issueDict['labels'] = issue['labels']
        if issue['milestone'] is not None:
            issueDict['milestone'] = issue['milestone']['title']
        else:
            issueDict['milestone'] = 'None'
        issuesList.append(issueDict)

td = datetime.datetime.today()
start_datetime =  td  + relativedelta(days=-7) 
recently_datetime = start_datetime.astimezone(pytz.timezone('US/Eastern'))

today = td.astimezone(pytz.timezone('US/Eastern'))
todayDate = today.strftime("%m/%d/%Y")
display(Markdown( '$ ' + reportTitle.replace('TDATE', todayDate  ) + ' $' ))

In [ ]:
#
# MILESTONE section
#
wrapper = textwrap.TextWrapper(width=80)
mStoneMatrix = ' \\begin{array}{| l l |} \\hline '
mStoneTitle =  '\\large { \\color{blue}{\\textbf{Currently Open Milestones} } }  \\\[1pt]  '

sortedStoneList = sorted(milestonesList ,key=lambda i:i['due_on'] )

for milestone in sortedStoneList:
#     print(milestone['title'])
    matrixLine = ' \\small { \\color{blue} {\\textbf {MSTONE}} } & \\small {\\textsf{Due: MDATE } }  \\\ '
    matrixLine = matrixLine.replace('MSTONE', milestone['title'] )
    dueDate = milestone['due_on'][5:10] if milestone['due_on'] else 'TBD'
    matrixLine = matrixLine.replace('MDATE',dueDate )
    matrixLine2 = '   MDESCR  & \\small \\textsf{ }   \\\ '
    mDescr = milestone['description'] if milestone['description'] else "None"
    # Wrap this text.
    mDescr = wrapper.wrap(text=mDescr)
    # Print each line.
    descrMatrix = ' \\hspace {10mm} \\begin{array}{ l}  '
    for element in mDescr:
        mLine =  ' \\small { \\textsf {MLINE} } \\\ '
        mLine = mLine.replace('MLINE',element)
        descrMatrix = descrMatrix + mLine
#         print(element)
    descrMatrix = descrMatrix + '  \\end{array} '
    matrixLine2 = matrixLine2.replace('MDESCR' ,  descrMatrix )
    mStoneMatrix = mStoneMatrix  +   matrixLine + matrixLine2 + ' \\hdashline  '

mStoneMatrix = mStoneMatrix + ' \\end{array} \\\[1pt]  '
display(Markdown( '$ \\hspace {60mm} ' + mStoneTitle + ' \\hspace {20mm} '  + mStoneMatrix + '  $' ))
# print( mStoneTitle + mStoneMatrix   )
# print('num: ', len(mDescr))

In [ ]:
#
#  REPO's Summary section
#
grandTotRepoOpen = 0
grandTotRepoClosed = 0

for repo in reposIssuesList: 
    grandTotRepoOpen = grandTotRepoOpen + repo['open']
    grandTotRepoClosed = grandTotRepoClosed + repo['closed']
    
repoMatrix = ' \\begin{array}{| l l |} \\hline '
newMatrixLine = ''

for repo in reposIssuesList: 
#     print(repo['repo_name'])
    repoMatrixLine = '\\hspace {0mm} \\small {\\textbf{REPO}  }  &  STATE  \\\[1pt] \\hdashline '
    newLine = repoMatrixLine.replace('REPO',repo['repo_name'])
    # added phantom to right justify all state matrixes
    repoState = STATE_TEMPLATE.replace('OPEN' ,' '+ ' \\phantom{1} \\textbf{' + str(repo['open']) + '} ')
    repoState = repoState.replace('CLOSED' , str(repo['closed'])  ) 
    newLine =  newLine.replace('STATE' , repoState )
    newMatrixLine = newMatrixLine + newLine
    
totStates = STATE_TEMPLATE.replace('OPEN' ,' \\textbf{' + str(grandTotRepoOpen) + '}' )
totStates = totStates.replace('CLOSED' , ' \\textbf{' + str(grandTotRepoClosed) + '}'  )
repoMatrixLine = '\\hspace {0mm} \\small {\\textbf{REPO}  }  &  STATE  \\\[1pt] \\hline '
newLine = repoMatrixLine.replace('REPO', 'Total')
newLine =  newLine.replace('STATE' , totStates )
newMatrixLine = newMatrixLine + newLine
repoMatrix = repoMatrix + newMatrixLine + '  \\end{array} \\\[1pt] '
# print(repoState)
repoTitle =  ' \\Huge {\\phantom{T}} \\large {\\color{black}{\\textbf{Repository Issue Status Summary}}} \\\[1pt]  '
display(Markdown( '$ \\hspace {52mm} ' + repoTitle + ' \\hspace {60mm}  ' + repoMatrix  + ' $' ))

In [ ]:
#
#  ISSUES section
#
currentIssuesTitle = """
$ 
\\hspace {80mm}  \\large \\color{red}  {\\textbf{Current Open Issues}} \\Huge {  \\phantom{X}} 
\\hspace {85mm} \\normalsize \\color{blue}  {\\textsf{Grouped by Milestone}} \\\[10pt]
$ """
display(Markdown(currentIssuesTitle ))
milestoneTitle = '$ \hspace {0mm} \\large  \\color{blue}{\\textsf{ MSTONE}} $ '

# Loop Through Milestones and display issues
for milestone in sortedStoneList:
    display(Markdown( milestoneTitle.replace('MSTONE', milestone['title']  ) ))
    for issue in issuesList:
        if issue['state'] == 'open' and issue['milestone'] == milestone['title']:
    #         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
            openIssues = OPEN_ISSUES_TEMPLATE
            openIssues = openIssues.replace('TITLE' , issue['issue_title'] )
            openIssues = openIssues.replace('NBR' , str(issue['issue_number']) )
            openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
            age = td - openedDate
            dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
    #         print('Opened ' + str(age.days) + ' days ago on ' + dateOpened )
    #         print('Module: ' + issue['repo_name']    )
            openIssues = openIssues.replace('AGE' , str(age.days ) )
            openIssues = openIssues.replace('ODATE' , dateOpened )
            openIssues = openIssues.replace('MODULE' , str(issue['repo_name']) )
    #  Display Tasks
            issueBody = issue['body']
            if issueBody is not None :
                issueCnt = 0
    #             issueLineList  = string2List(issueBody)
                issueMatrix = ' \\begin{array}{l}  '
                for issueLine in issueBody:
                    if len(issueLine) > 1 and ('[ ]' in  issueLine or '[x]' in issueLine) :
                        issueCnt += 1
                        matrixLine = ' \\small {  CHECKBOX \\ \\textsf{LINE }  } \\\ '
                        checkbox = issueLine[:5]
                        checkbox = checkbox.replace('- [x]'  , ' \\color{green}{\\boxed{x}} \\ '  )
                        checkbox = checkbox.replace('- [ ]'  , ' \\color{red}{\\boxed{\\phantom{x}}} \\ '  )
                        matrixLine = matrixLine.replace( 'CHECKBOX'  , checkbox )
                        issueMatrix = issueMatrix + matrixLine.replace('LINE', issueLine[6:]   )
                if issueCnt > 0:
                    issueMatrix = issueMatrix + '  \\end{array}  '
                    tasksComments = TASKS_TEMPLATE
                    tasksComments = tasksComments.replace('TASKS' ,issueMatrix)
                    openIssues = openIssues.replace('TASKS' , tasksComments )
                else:
                    openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )
            else:
                openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )  
    # Display Comments
            if len(issue['labels']) > 0:
                labelsMatrix = ' \\begin{array}{l}    '
                for label in issue['labels'] :
                    matrixLine = ' \\small { \\textsf{LABEL }}  \\\  '
                    labelsMatrix = labelsMatrix + matrixLine.replace('LABEL', label['description'] )
                labelsMatrix = labelsMatrix + '  \\end{array}   '
                issueComments = LABELS_TEMPLATE
                openIssues = openIssues.replace('LABELS' , issueComments.replace('COMM',labelsMatrix ))
            else:
                openIssues = openIssues.replace('LABELS' , ' \\textit{No Comments} ' )    

            display(Markdown ('$ ' + openIssues   + ' $'    ))

In [ ]:
#
# CLOSED ISSUES SECTION
#

display(Markdown( closedTitle  ))
for issue in issuesList:
    if issue['state'] == 'closed':
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        closedDate = datetime.datetime.strptime(issue['date_closed'],"%Y-%m-%dT%H:%M:%SZ")
        dateClosed =closedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        age = td - closedDate
        if age.days > 7 :
            continue
        closedIssue = CLOSED_ISSUE_TEMPLATE
        closedIssue = closedIssue.replace('TITLE' , issue['issue_title'] )
        closedIssue = closedIssue.replace('NBR' , str(issue['issue_number']) )            
        closedIssue = closedIssue.replace('AGE' , str(age.days ) )
        closedIssue = closedIssue.replace('ODATE' , dateOpened )
        closedIssue = closedIssue.replace('CDATE' , dateClosed )
        closedIssue = closedIssue.replace('MODULE' , str(issue['repo_name']) )       
        if len(issue['labels']) > 0:
            labelsMatrix = ' \\begin{array}{l}    '
            for label in issue['labels'] :
                matrixLine = ' \\small { \\textsf{LABEL }}  \\\  '
                labelsMatrix = labelsMatrix + matrixLine.replace('LABEL', label['description'] )
            labelsMatrix = labelsMatrix + '  \\end{array}   '
            issueComments = LABELS_TEMPLATE
            closedIssue = closedIssue.replace('LABELS' , issueComments.replace('COMM',labelsMatrix ))
        else:
            closedIssue = closedIssue.replace('LABELS' , ' \\textit{No Comments} ' )    
            
        display(Markdown ('$ ' + closedIssue   + ' $'    ))

In [ ]:
#
# RECENTLY OPEN ISSUES SECTION
#

display(Markdown( openedTitle ))
for issue in issuesList:
    if issue['state'] == 'open':
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        age = td - openedDate
        if age.days > 7 :
            continue
#         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        
        openIssue = ISSUE_TEMPLATE
        openIssue = openIssue.replace('TITLE' , issue['issue_title'] )
        openIssue = openIssue.replace('NBR' , str(issue['issue_number']) )            
        openIssue = openIssue.replace('AGE' , str(age.days ) )
        openIssue = openIssue.replace('ODATE' , dateOpened )
        openIssue = openIssue.replace('MODULE' , str(issue['repo_name']) )            
        display(Markdown ('$ ' + openIssue   + ' $'    ))       

print('-------------')